# PySpark homework assignment

## Context

The goal of this assignment is to get view on your coding workflow & style.  Your main focus should be creating performant & robust code for data manipulations.  

For a homework assignment, we cannot grant you access to our infrastructure (Cloudera data platform on prem: a spark cluster deployment on Yarn).  Since the focus is on development, we provided a template notebook to get up and running very quickly on Google Colab.  

You have the freedom to perform this assignment on any spark3+ infrastructure.  If want to use a local or cloud setup, go for it!

Some of the tasks are open for interpretation.  This allows us to assess business understanding and relevant field experience.  These tasks are not pass or fail checks.  During the interview we'll ask details about the choice(s) you made.

For the assignment, you'll be working with store location data.  You might be familiar with the phrase "Location, location, location" from the real-estate context.  The same house can have a different selling price based on the location.  In fast moving consumer goods (FMCG), location is one of the most crucial aspects:

* Proximity & accessibility to customers increases convenience
* Proximity to competitors increases market pressure
* It has impact on the supply chain

## Evaluation criteria

1. Software engineering
   1. Clean code (e.g. using meaningful names)
   1. Robust & efficient code
   1. Styling (e.g. PEP8, or Google style guide)
   1. Documentation(e.g. docstrings)
   1. Design (e.g. SOLID principles)
1. Workflow
   1. How you use Git
   1. How you structure your assignment
   1. Owning mistakes
   1. Rationale for design decisions
   1. Making your solution accessible to others
1. Business context
   1. GDPR
   1. Fast moving consumer goods
1.(optional: own infra) System engineering
   1. What setup did you use?
   1. How did you set it up?

## Deliverables we expect

1. Private GitHub repo
   1. Colab allows you to save to GitHub
   1. Invite my username to your private repo as contributor
1. README.md with relevant content
1. Code relevant to the assignment


## Google colab spark setup

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
from os import environ
import findspark

In [ ]:
# Setting environment variables
environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
# Init spark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
# spark.sql.repl.eagerEval.enabled: Property used to format output tables better

spark = (
    SparkSession
    .builder
    .appName("cg-pyspark-assignment")
    .master("local")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
  )

spark

## Getting the assignment data

This will call the api and save the results in current working directory as .json files

In [ ]:
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places > clp-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/okay-places > okay-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/spar-places > spar-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/dats-places > dats-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/cogo-colpnts > cogo-places.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  216k    0  216k    0     0   174k      0 --:--:--  0:00:01 --:--:--  174k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  143k    0  143k    0     0   147k      0 --:--:-- --:--:-- --:--:--  147k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  167k    0  167k    0     0   171k      0 --:--:-- --:--:-- --:--:--  171k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89162    0 89162    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
!pwd

/content


## Assignment instructions

1. Download the data from api
1. Create a logger object that logs to a file "assignment.log"
   1. You can add whatever logging config you want or need
   1. At least on Filehandler based on instructions
1. implement get_data_by_brand function
   1. Follow instructions in docstring
   1. df_clp code line should work
1. No more handholding ... :-)
1. Create a single object (dataframe) that:
   1. Contains data from **all brands**
      1. Not every brand has the same columns!
   1. Drop placeSearchOpeningHours
   1. You can keep sellingPartners as an array
   1. Extract "postal_code" from address
   1. Create new column "province" derived from postal_code
   1. Transform geoCoordinates into lat and lon column
   1. One-hot-encode the handoverServices
   1. Pretend houseNumber and streetName are GDPR sensitive.
      1. How would you anonymize this data for unauthorized users?
      1. (optional) Implement the above
      1. How would you show the real data to authorized users?
      1. (optional) Implement the above
1. Save the end result as a parquet file
   1. (optional)partitioning?

**postal_code** logic:
* "Brussel": 1000-1299  
* "Waals-Brabant": 1300-1499  
* "Vlaams-Brabant": 1500-1999, 3000-3499  
* "Antwerpen": 2000-2999  
* "Limburg": 3500-3999  
* "Luik": 4000-4999  
* "Namen": 5000-5999  
* "Henegouwen": 6000-6599,7000-7999  
* "Luxemburg": 6600-6999  
* "West-Vlaanderen": 8000-8999  
* "Oost-Vlaanderen": 9000-9999

In [ ]:
# Import statements should go here
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, regexp_extract
from pyspark.sql.functions import array_contains, col
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, LongType, DoubleType, StructType
import logging

In [ ]:
# Define schema
schema = StructType([
    StructField("address", StringType(), True),
    StructField("branchId", StringType(), True),
    StructField("commercialName", StringType(), True),
    StructField("ensign", StringType(), True),
    StructField("geoCoordinates", StructType([
        StructField("latitude", DoubleType(), True),
        StructField("longitude", DoubleType(), True)
    ]), True),
    StructField("handoverServices", ArrayType(StringType()), True),
    StructField("isActive", StringType(), True),
    StructField("moreInfoUrl", StringType(), True),
    StructField("placeId", LongType(), True),
    StructField("placeSearchOpeningHours", ArrayType(StructType([
        StructField("from", StringType(), True),
        StructField("till", StringType(), True)
    ])), True),
    StructField("placeType", StringType(), True),
    StructField("routeUrl", StringType(), True),
    StructField("sellingPartners", ArrayType(StringType()), True),
    StructField("sourceStatus", StringType(), True),
    StructField("temporaryClosures", ArrayType(StringType()), True),
])


In [ ]:
# Configure logging
logging.basicConfig(filename='assignment.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
LOGGER = getLogger()

In [ ]:
def normalize_handover_services(df):
    if 'handoverServices' in df.columns:
        df = df.withColumn("handoverServices", col("handoverServices").cast(ArrayType(StringType())))
    else:
        df = df.withColumn("handoverServices", lit(None).cast(ArrayType(StringType())))
    return df

def get_data_by_brand(brand: str, logger=logging.getLogger()):
    """Fetch input data based on brand."""
    file_path = f'{brand}-places.json'
    try:
        df = spark.read.schema(schema).json(file_path)
        if df.count() == 0:
            logger.warning(f"No data found for brand: {brand}")
        df = df.withColumn('brand', lit(brand))
        df = normalize_handover_services(df)
        logger.info(f"Data loaded successfully for brand: {brand}")
        return df
    except Exception as e:
        logger.error(f"Error loading data for brand: {brand}, Error: {str(e)}")
        raise e

In [ ]:
# Load data for all brands
brands = ["clp", "okay", "spar", "dats", "cogo"]
dfs = [get_data_by_brand(brand) for brand in brands]

In [ ]:
# Combine dataframes
combined_df = dfs[0]
for df in dfs[1:]:
    combined_df = combined_df.unionByName(df, allowMissingColumns=True)

# Drop placeSearchOpeningHours
combined_df = combined_df.drop("placeSearchOpeningHours")

# Extract postal_code
combined_df = combined_df.withColumn("postal_code", regexp_extract(col("address"), r"\d{4}", 0))

# Define postal code to province mapping
postal_code_province = [
    (1000, 1299, "Brussel"),
    (1300, 1499, "Waals-Brabant"),
    (1500, 1999, "Vlaams-Brabant"),
    (3000, 3499, "Vlaams-Brabant"),
    (2000, 2999, "Antwerpen"),
    (3500, 3999, "Limburg"),
    (4000, 4999, "Luik"),
    (5000, 5999, "Namen"),
    (6000, 6599, "Henegouwen"),
    (7000, 7999, "Henegouwen"),
    (6600, 6999, "Luxemburg"),
    (8000, 8999, "West-Vlaanderen"),
    (9000, 9999, "Oost-Vlaanderen")
]

def get_province(postal_code):
    code = int(postal_code)
    for start, end, province in postal_code_province:
        if start <= code <= end:
            return province
    return None

get_province_udf = udf(get_province, StringType())

# Add province column
combined_df = combined_df.withColumn("province", get_province_udf(col("postal_code")))

# Transform geoCoordinates into lat and lon columns
combined_df = combined_df.withColumn("latitude", col("geoCoordinates.latitude"))
combined_df = combined_df.withColumn("longitude", col("geoCoordinates.longitude"))

# One-hot encode the handoverServices
distinct_services = combined_df.selectExpr("explode(handoverServices)").distinct().collect()
for service in distinct_services:
    service_name = service[0]
    combined_df = combined_df.withColumn(service_name, array_contains(col("handoverServices"), service_name))

# Save the end result as a parquet file
combined_df.write.parquet("processed_data.parquet")